In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
import string

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from scipy import stats
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc, roc_curve
from sklearn.metrics import f1_score, classification_report

from sklearn import svm
import gensim.models as g
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv('/Users/daviddeng8/Documents/Projects/affinity/data/preprocessed_cleaned_body.csv')

In [3]:
df

,Body,Bias,cleaned_body
0,Abortion rights advocates have asked the U.S. ...,1.67,abortion right advocate asked u supreme court ...
1,A federal appeals court rejected the most dire...,0.67,federal appeal court rejected direct constitut...
2,As part of the Trump administration's effort t...,-2.75,part trump administration effort slow migrant ...
3,"President Donald Trump and ""the Trump of the T...",-4.33,president donald trump trump tropic brazilian ...
4,"U.S Senator Elizabeth Warren, who is competing...",-10.00,u senator elizabeth warren competing democrati...
...,...,...,...
1670,Only hours after the first flames that eventua...,19.75,hour first flame eventually would consume notr...
1671,Washington watchdog Judicial Watch believes th...,30.50,washington watchdog judicial watch belief fbi ...
1672,"Two Fox News personalities, as Notre Dame Cath...",29.75,two fox news personality notre dame cathedral ...
1673,Chase Bank is shutting down accounts of people...,22.00,chase bank shutting account people organizatio...


In [4]:
right = df.loc[df['Bias'] > 0]
left = df.loc[df['Bias'] <= 0]

# 1 = Right bias, 0 = Left bias
right['Bias'] = 1
left['Bias'] = 0

cat_df = pd.concat([right, left])
cat_df

/Users/daviddeng8/anaconda2/envs/practice/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/daviddeng8/anaconda2/envs/practice/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Body,Bias,cleaned_body
0,Abortion rights advocates have asked the U.S. ...,1,abortion right advocate asked u supreme court ...
1,A federal appeals court rejected the most dire...,1,federal appeal court rejected direct constitut...
7,Michael Cohen has been officially disbarred in...,1,michael cohen ha officially disbarred new york...
13,President Donald Trump has vetoed a resolution...,1,president donald trump ha vetoed resolution st...
15,Newly-released body camera footage reveals the...,1,newlyreleased body camera footage reveals dram...
...,...,...,...
1657,THE DAY OF THE BIGGEST COVER-UP OF ALL IS NIGH...,0,day biggest coverup nigh thursday anticipating...
1658,"If you are a man or a lady of letters, and are...",0,man lady letter also versed internet tradition...
1659,Attorney General William Barr would like to re...,0,attorney general william barr would like remin...
1661,"(PASTEMAGAZINE) — Monday night, Bernie Sanders...",0,pastemagazine — monday night bernie sander tow...


In [5]:
X = cat_df.iloc[:, cat_df.columns != 'Bias']
Y = cat_df['Bias']

In [6]:
X

,Body,cleaned_body
0,Abortion rights advocates have asked the U.S. ...,abortion right advocate asked u supreme court ...
1,A federal appeals court rejected the most dire...,federal appeal court rejected direct constitut...
7,Michael Cohen has been officially disbarred in...,michael cohen ha officially disbarred new york...
13,President Donald Trump has vetoed a resolution...,president donald trump ha vetoed resolution st...
15,Newly-released body camera footage reveals the...,newlyreleased body camera footage reveals dram...
...,...,...
1657,THE DAY OF THE BIGGEST COVER-UP OF ALL IS NIGH...,day biggest coverup nigh thursday anticipating...
1658,"If you are a man or a lady of letters, and are...",man lady letter also versed internet tradition...
1659,Attorney General William Barr would like to re...,attorney general william barr would like remin...
1661,"(PASTEMAGAZINE) — Monday night, Bernie Sanders...",pastemagazine — monday night bernie sander tow...


In [7]:
Y

0       1
1       1
7       1
13      1
15      1
       ..
1657    0
1658    0
1659    0
1661    0
1668    0
Name: Bias, Length: 1675, dtype: int64

In [8]:
bow_transformer = CountVectorizer(max_features = 800)
bow = bow_transformer.fit_transform(X['cleaned_body'])

X = pd.DataFrame(bow.toarray(), columns=bow_transformer.get_feature_names())
X

,ability,able,abortion,abuse,access,according,account,accused,across,act,...,would,wrong,wrote,year,yearold,yet,york,you,young,zealand
0,0,0,11,0,2,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,3,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,3,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,2,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670,0,0,0,0,0,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0
1671,0,0,0,0,0,0,0,0,0,0,...,3,1,0,0,0,0,0,0,0,0
1672,0,0,0,1,0,0,0,0,0,1,...,4,0,1,2,0,0,1,0,0,0
1673,0,1,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0


In [10]:
col_names = X.columns

scaler = MinMaxScaler()
scaled = scaler.fit_transform(X)
X = pd.DataFrame(scaled, columns=col_names)

In [11]:
X

,ability,able,abortion,abuse,access,according,account,accused,across,act,...,would,wrong,wrote,year,yearold,yet,york,you,young,zealand
0,0.0,0.00,0.52381,0.000000,0.2,0.000000,0.0,0.0,0.142857,0.000000,...,0.018182,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
1,0.0,0.00,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.153846,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
2,0.0,0.00,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.018182,0.000000,0.000000,0.000000,0.0,0.0,0.130435,0.000000,0.000000,0.0
3,0.0,0.00,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.058824,...,0.000000,0.000000,0.000000,0.076923,0.0,0.0,0.000000,0.000000,0.000000,0.0
4,0.0,0.00,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.025641,0.4,0.0,0.000000,0.066667,0.111111,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670,0.0,0.00,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.054545,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
1671,0.0,0.00,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.054545,0.166667,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
1672,0.0,0.00,0.00000,0.071429,0.0,0.000000,0.0,0.0,0.000000,0.058824,...,0.072727,0.000000,0.076923,0.051282,0.0,0.0,0.043478,0.000000,0.000000,0.0
1673,0.0,0.25,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.036364,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=1)

In [13]:
model = svm.SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print ("accuracy score: ", accuracy_score(y_test, y_pred))

accuracy score:  0.7136038186157518


In [14]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.99      0.83       290
           1       0.85      0.09      0.15       129

    accuracy                           0.71       419
   macro avg       0.78      0.54      0.49       419
weighted avg       0.75      0.71      0.62       419



In [ ]:
X_train

In [ ]:
y_train

### TfIdf 

In [28]:
'''vectorizer = TfidfVectorier(use_idf=True, stop_words='english')
matrix = vectorizer.fit_transform()'''
X_tfidf = cat_df['cleaned_body']
Y_tfidf = cat_df['Bias']

In [23]:
vectorizer = TfidfVectorizer(use_idf = True, stop_words = 'english')

In [29]:
matrix = vectorizer.fit_transform(X_tfidf)

In [31]:
df = pd.DataFrame(matrix.toarray(), columns=vectorizer.get_feature_names())
#X = pd.DataFrame(bow.toarray(), columns=bow_transformer.get_feature_names())
df

,aa,aaa,aaf,aafs,aaliyah,aapl,aaron,aaronblake,aarp,aarseth,...,égalité,électricité,élysée,émigré,état,île,øystein,überrich,𝙹𝚞𝚊𝚗,𝚁𝚘𝚍𝚛𝚒𝚐𝚞𝚎𝚣
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1671,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
col_names = df.columns

scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)
df = pd.DataFrame(scaled, columns=col_names)

In [33]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(df, Y_tfidf, random_state=1)

In [38]:
model = svm.SVC()
print ("hi")
model.fit(X_train2, y_train2)
print ("2")
y_pred = model.predict(X_test2)
print ("3")
print ("accuracy score: ", accuracy_score(y_test2, y_pred))

hi
2
3
accuracy score:  0.6992840095465394


### SVM with Doc2Vec

In [39]:
cat_df

,Body,Bias,cleaned_body
0,Abortion rights advocates have asked the U.S. ...,1,abortion right advocate asked u supreme court ...
1,A federal appeals court rejected the most dire...,1,federal appeal court rejected direct constitut...
7,Michael Cohen has been officially disbarred in...,1,michael cohen ha officially disbarred new york...
13,President Donald Trump has vetoed a resolution...,1,president donald trump ha vetoed resolution st...
15,Newly-released body camera footage reveals the...,1,newlyreleased body camera footage reveals dram...
...,...,...,...
1657,THE DAY OF THE BIGGEST COVER-UP OF ALL IS NIGH...,0,day biggest coverup nigh thursday anticipating...
1658,"If you are a man or a lady of letters, and are...",0,man lady letter also versed internet tradition...
1659,Attorney General William Barr would like to re...,0,attorney general william barr would like remin...
1661,"(PASTEMAGAZINE) — Monday night, Bernie Sanders...",0,pastemagazine — monday night bernie sander tow...


In [40]:
text = cat_df['cleaned_body']
new_df = cat_df.drop('Body', axis=1)
#ls = [body.split() for body in text]
new_df

,Bias,cleaned_body
0,1,abortion right advocate asked u supreme court ...
1,1,federal appeal court rejected direct constitut...
7,1,michael cohen ha officially disbarred new york...
13,1,president donald trump ha vetoed resolution st...
15,1,newlyreleased body camera footage reveals dram...
...,...,...
1657,0,day biggest coverup nigh thursday anticipating...
1658,0,man lady letter also versed internet tradition...
1659,0,attorney general william barr would like remin...
1661,0,pastemagazine — monday night bernie sander tow...


In [42]:
#ls = [body.split(), rating for body, rating in new_df]

SyntaxError: invalid syntax (<ipython-input-42-91ca632c7f1b>, line 1)

In [ ]:
#newX_train, newX_test, newy_train, newy_test = train_test_split(X, Y, random_state=1)

In [41]:
cat_df

,Body,Bias,cleaned_body
0,Abortion rights advocates have asked the U.S. ...,1,abortion right advocate asked u supreme court ...
1,A federal appeals court rejected the most dire...,1,federal appeal court rejected direct constitut...
7,Michael Cohen has been officially disbarred in...,1,michael cohen ha officially disbarred new york...
13,President Donald Trump has vetoed a resolution...,1,president donald trump ha vetoed resolution st...
15,Newly-released body camera footage reveals the...,1,newlyreleased body camera footage reveals dram...
...,...,...,...
1657,THE DAY OF THE BIGGEST COVER-UP OF ALL IS NIGH...,0,day biggest coverup nigh thursday anticipating...
1658,"If you are a man or a lady of letters, and are...",0,man lady letter also versed internet tradition...
1659,Attorney General William Barr would like to re...,0,attorney general william barr would like remin...
1661,"(PASTEMAGAZINE) — Monday night, Bernie Sanders...",0,pastemagazine — monday night bernie sander tow...


In [ ]:
documents = [TaggedDocument(doc.split(), [i]) for doc in cat_df['cleaned_body'] for i in cat_df['Bias']]

In [ ]:
documents

In [ ]:
pred = model.infer_vector(ls[0])